<a href="https://colab.research.google.com/github/wolfram-roemhild/dcgan/blob/master/start.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!git clone https://github.com/wolfram-roemhild/dcgan start
!ls